In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np

In [ ]:
t = tf.constant(2)
t

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [ ]:
t.numpy()

2

In [ ]:
np.array(t)

array(2, dtype=int32)

In [ ]:
t1 = tf.constant([[2,3], [4,5]])
t1

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2, 3],
       [4, 5]], dtype=int32)>

Element-wise multiplication

In [ ]:
t2 = t*t1
t2

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 4,  6],
       [ 8, 10]], dtype=int32)>

In [ ]:
t3 = t + t1
t3

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[4, 5],
       [6, 7]], dtype=int32)>

Matrix multiplication

Tesnorler immutable yani değerlerinin değişemediği bir yapıdadır. Makine öğrenmesi sırasında mutable yapılara ihtiyacımız var.

In [ ]:
t4 = t1 @ t2
t4

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[32, 42],
       [56, 74]], dtype=int32)>

In [ ]:
v = tf.Variable(t1, name="var")
v

<tf.Variable 'var:0' shape=(2, 2) dtype=int32, numpy=
array([[2, 3],
       [4, 5]], dtype=int32)>

In [ ]:
v.assign([[6,5],[4,3]])
v

<tf.Variable 'var:0' shape=(2, 2) dtype=int32, numpy=
array([[6, 5],
       [4, 3]], dtype=int32)>

In [ ]:
v.assign_add([[2,3],[6,7]])
v

<tf.Variable 'var:0' shape=(2, 2) dtype=int32, numpy=
array([[ 8,  8],
       [10, 10]], dtype=int32)>

In [ ]:
v1 = tf.Variable(0, trainable=False)
for _ in range(10):
  v1.assign_add(1)

v1

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=10>

In [ ]:
x = tf.Variable([[2,3,4]])
weights = tf.Variable(tf.random.normal((3,2)))
bias = tf.zeros((2,))

x, weights, bias

(<tf.Variable 'Variable:0' shape=(1, 3) dtype=int32, numpy=array([[2, 3, 4]], dtype=int32)>,
 <tf.Variable 'Variable:0' shape=(3, 2) dtype=float32, numpy=
 array([[-0.614495  , -0.36395827],
        [-0.18979234,  1.2019783 ],
        [ 1.0963964 , -0.04740789]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0., 0.], dtype=float32)>)

In [ ]:
class P:
  def __init__(self,y):
    self.y = y

  def __str__(self):
    return f"{self.x} - {self.y}"

  @property
  def x(self):
    return 1.0

p=P(2.0)

print(p)

1.0 - 2.0


4

In [ ]:
#######################################
# Hitters
#######################################

# !pip install xgboost
# !pip install lightgbm
# conda install -c conda-forge lightgbm
# !pip install catboost

import warnings
warnings.simplefilter(action='ignore', category=Warning)
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate

from helpers.data_preparation import *
from helpers.exploratory_data_analysis import *


df = pd.read_csv("datasets/hitters.csv")
df.head()

#######################################
# Quick Data Preprocessing
#######################################

df.dropna(inplace=True)
cat_cols, num_cols, cat_but_car = grab_col_names(df)

for col in cat_cols:
    cat_summary(df, col)

for col in cat_cols:
    df.loc[:, col] = label_encoder(df, col)

check_df(df)

y = df["Salary"]
X = df.drop(["Salary"], axis=1)

######################################################
# Base Models
######################################################

models = [('LR', LinearRegression()),
          ("Ridge", Ridge()),
          ("Lasso", Lasso()),
          ("ElasticNet", ElasticNet()),
          ('KNN', KNeighborsRegressor()),
          ('CART', DecisionTreeRegressor()),
          ('RF', RandomForestRegressor()),
          ('SVR', SVR()),
          ('GBM', GradientBoostingRegressor()),
          ("XGBoost", XGBRegressor(objective='reg:squarederror')),
          ("LightGBM", LGBMRegressor()),
          # ("CatBoost", CatBoostRegressor(verbose=False))
          ]

for name, regressor in models:
    rmse = np.mean(np.sqrt(-cross_val_score(regressor, X, y, cv=10, scoring="neg_mean_squared_error")))
    print(f"RMSE: {round(rmse, 4)} ({name}) ")


######################################################
# Automated Hyperparameter Optimization
######################################################


cart_params = {'max_depth': range(1, 20),
               "min_samples_split": range(2, 30)}

rf_params = {"max_depth": [5, 8, 15, None],
             "max_features": [5, 7, "auto"],
             "min_samples_split": [8, 15, 20],
             "n_estimators": [200, 500, 1000]}

xgboost_params = {"learning_rate": [0.1, 0.01, 0.01],
                  "max_depth": [5, 8, 12, 20],
                  "n_estimators": [100, 200, 300, 500],
                  "colsample_bytree": [0.5, 0.8, 1]}

lightgbm_params = {"learning_rate": [0.01, 0.1, 0.001],
                   "n_estimators": [300, 500, 1500],
                   "colsample_bytree": [0.5, 0.7, 1]}

regressors = [("CART", DecisionTreeRegressor(), cart_params),
              ("RF", RandomForestRegressor(), rf_params),
              ('XGBoost', XGBRegressor(objective='reg:squarederror'), xgboost_params),
              ('LightGBM', LGBMRegressor(), lightgbm_params)]

best_models = {}

for name, regressor, params in regressors:
    print(f"########## {name} ##########")
    rmse = np.mean(np.sqrt(-cross_val_score(regressor, X, y, cv=10, scoring="neg_mean_squared_error")))
    print(f"RMSE: {round(rmse, 4)} ({name}) ")

    gs_best = GridSearchCV(regressor, params, cv=3, n_jobs=-1, verbose=False).fit(X, y)

    final_model = regressor.set_params(**gs_best.best_params_)
    rmse = np.mean(np.sqrt(-cross_val_score(final_model, X, y, cv=10, scoring="neg_mean_squared_error")))
    print(f"RMSE (After): {round(rmse, 4)} ({name}) ")

    print(f"{name} best params: {gs_best.best_params_}", end="\n\n")

    best_models[name] = final_model


######################################################
# # Stacking & Ensemble Learning
######################################################

voting_reg = VotingRegressor(estimators=[('RF', best_models["RF"]),
                                         ('LightGBM', best_models["LightGBM"])])

voting_reg.fit(X, y)


np.mean(np.sqrt(-cross_val_score(voting_reg, X, y, cv=10, scoring="neg_mean_squared_error")))

######################################################
# Prediction for a New Observation
######################################################

X.columns
random_user = X.sample(1, random_state=45)
voting_reg.predict(random_user)
